<a href="https://colab.research.google.com/github/cras-lab/ChatGPT/blob/main/RAG_Toy_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FAISS(Facebook AI Similarity Search)는 Facebook AI Research에서 개발한 라이브러리로, 벡터를 효율적으로 검색하고 인덱싱하기 위한 도구이다. 주로 대규모 벡터 데이터베이스에서 유사한 벡터를 빠르게 검색하는 데 사용된다.<BR>
먼저 faiss를 설치해야 한다.

In [1]:
pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.9 MB/s eta 0:00:00


필요한 라이브러리를 임포트 한다.

In [2]:
# 라이브러리 임포트
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import faiss
import numpy as np

RAG의 핵심인 추가적인 검색 데이터를 구축한다. <BR>
여기서는 예제로서 간단한 문장 몇개를 벡터화 한다.

In [3]:
# 1. 검색에 사용할 문서 데이터
documents = [
    "The Eiffel Tower is located in Paris.",
    "The Great Wall of China is one of the seven wonders of the world.",
    "The Moon orbits around the Earth.",
    "Python is a popular programming language for machine learning."
    ]

간단한 임베딩 생성을 위해 TF-IDF를 사용해서 문서를 임베딩한다.
임베딩을 위한 함수를 정의한다.

In [4]:
# 2. 간단한 임베딩 생성을 위해 TF-IDF 사용
# 문서를 임베딩하는 함수
def embed_documents(documents):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    embeddings = vectorizer.fit_transform(documents)
    return embeddings.toarray(), vectorizer

문서를 임베딩하고 ,검색을 위한 인덱스 생성

In [5]:
# 3. 문서를 임베딩하고, 검색을 위한 인덱스를 생성한다.
doc_embeddings, vectorizer = embed_documents(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

검색과 생성 파이프라인을 구축한다.

In [21]:
# 4. 검색 및 생성 파이프라인 구축
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
qa_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

사용자의 쿼리에 기반한 문서검색 함수를 정의

In [22]:
# 5. 사용자의 쿼리를 기반으로 문서 검색
def search_documents(query, k=1):
    query_embedding = vectorizer.transform([query]).toarray()
    _, indices = index.search(query_embedding, k)
    return [documents[i] for i in indices[0]]

검색된 문서에 기반한 답변생성 함수 정의

In [23]:
# 6. 검색된 문서를 기반으로 답변 생성
def generate_answer(query):
    retrieved_docs = search_documents(query)
    input_text = " ".join(retrieved_docs) + " " + query

    answer = qa_pipeline(input_text, max_length=50, min_length=25, do_sample=False)
    return answer[0]['summary_text']

In [ ]:
# 7. 예제 실행
query = "Where is the Eiffel Tower?"
answer = generate_answer(query)
print(f"Query: {query}\nAnswer: {answer}")


이제 잘못된 정보를 제공했을 때의 결과를 보기 위해 검색 사용데이터를 의도적으로 왜곡해 본다.에펠탑이 서울에 있다고 거짓된 문서를 넣어본다.

In [25]:
# 1. 검색에 사용할 문서 데이터
documents = [
    "The Eiffel Tower is located in Seoul.",
    "The Great Wall of China is one of the seven wonders of the world.",
    "The Moon orbits around the Earth.",
    "Python is a popular programming language for machine learning."
    ]

다시 임베딩하고 파이프 라인을 구축해 본다.

In [26]:
# 3. 문서를 임베딩하고, 검색을 위한 인덱스를 생성한다.
doc_embeddings, vectorizer = embed_documents(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 4. 검색 및 생성 파이프라인 구축
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
qa_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

쿼리를 실행해 본다.

In [ ]:
# 7. 예제 실행
query = "Where is the Eiffel Tower?"
answer = generate_answer(query)
print(f"Query: {query}\nAnswer: {answer}")